<a href="https://colab.research.google.com/github/dumpling2/steamhistory/blob/main/Steam%E3%82%B2%E3%83%BC%E3%83%A0%E3%83%AA%E3%82%B9%E3%83%88%E5%8F%96%E5%BE%97%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Steam APIキーとSteamIDの取得方法 ---
# 1. Steam APIキー:
#    - Steam Developersポータル (https://steamcommunity.com/dev/apikey) にアクセス
#    - Steamアカウントでログイン
#    - "Register for a Steam Web API Key" をクリック
#    - ドメイン名を入力 (任意)
#    - 利用規約に同意して "Register" をクリック
#    - 表示された "Key" がSteam APIキーです
# 2. SteamID64:
#    - SteamプロフィールページのURLから確認できます。
#    - 例：https://steamcommunity.com/profiles/76561198000000001/
#    - 上記のURLの場合、SteamID64は "76561198000000001" です。
# ---

import requests
import json
import sys
import datetime
import time
import os
from contextlib import contextmanager
from google.colab import userdata

# APIリクエストのエラー処理を共通化
def handle_api_error(response, api_name, app_id=None):
    """APIリクエストのエラー処理を共通化"""
    try:
        response.raise_for_status()
        return response.json()
    except requests.exceptions.Timeout:
        message = f"エラー: {api_name} APIへの接続がタイムアウトしました。"
        if app_id:
            message = f"警告: AppID {app_id}: {api_name} API タイムアウト"
        print(message, file=sys.stderr)
        return None
    except requests.exceptions.HTTPError as e:
        message = f"エラー: {api_name} APIでHTTPエラーが発生しました ({e.response.status_code})."
        if app_id:
            if e.response.status_code == 404:
                message = f"情報: AppID {app_id}: {api_name} が見つかりません (404)."
            elif e.response.status_code == 429:
                message = f"警告: AppID {app_id}: {api_name} API レート制限 (429)."
            else:
                message = f"警告: AppID {app_id}: {api_name} API HTTPエラー ({e.response.status_code})."
        print(message, file=sys.stderr)
        return None
    except requests.exceptions.RequestException as e:
        message = f"エラー: {api_name} APIリクエスト中に問題が発生しました: {e}"
        if app_id:
            message = f"警告: AppID {app_id}: {api_name} API リクエストエラー: {e}"
        print(message, file=sys.stderr)
        return None
    except json.JSONDecodeError:
        message = f"エラー: {api_name} APIレスポンスのJSON解析に失敗しました。"
        if app_id:
            message = f"警告: AppID {app_id}: {api_name} API JSONデコードエラー"
        print(message, file=sys.stderr)
        return None

# --- APIリクエスト関数 ---
def get_owned_games(api_key, steam_id, include_appinfo=True, include_played_free_games=True):
    """Steam APIにリクエストを送信し、所有ゲームの基本データを取得する関数"""
    api_url = "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/"
    params = {
        "key": api_key,
        "steamid": steam_id,
        "format": "json",
        "include_appinfo": "1" if include_appinfo else "0",
        "include_played_free_games": "1" if include_played_free_games else "0",
    }
    print("GetOwnedGames APIに接続中...")
    response = requests.get(api_url, params=params, timeout=30)
    return handle_api_error(response, "GetOwnedGames")

def get_app_details(app_id):
    """指定されたAppIDのゲーム詳細データを取得する関数"""
    api_url = "https://store.steampowered.com/api/appdetails/"
    params = {
        "appids": app_id,
        "cc": "jp",
        "l": "japanese"
    }
    response = requests.get(api_url, params=params, timeout=15)
    data = handle_api_error(response, "App Details", app_id)
    if data and str(app_id) in data and data[str(app_id)].get("success"):
        return data[str(app_id)].get("data")
    return None

# --- ヘルパー関数 ---
def format_timestamp(timestamp):
    """Unixタイムスタンプをローカルの日時文字列に変換する"""
    return "プレイ記録なし" if timestamp is None or timestamp == 0 else datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

@contextmanager
def redirect_stdout(filepath):
    """標準出力を一時的にファイルにリダイレクトするコンテキストマネージャ"""
    original_stdout = sys.stdout
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            sys.stdout = f
            yield
    finally:
        sys.stdout = original_stdout

# --- メイン処理 ---
def main():
    is_colab = 'google.colab' in sys.modules
    api_key = userdata.get('API_KEY') if is_colab else os.environ.get('STEAM_API_KEY')
    steam_id = userdata.get('STEAM_ID') if is_colab else os.environ.get('STEAM_ID')

    if not api_key or not steam_id:
        print("エラー: APIキーまたはSteamIDが設定されていません。", file=sys.stderr)
        print("Google Colabで実行する場合は、以下のコードを実行してuserdataに設定してください:", file=sys.stderr)
        print("""
from google.colab import userdata
userdata.set('API_KEY', 'YOUR_API_KEY')
userdata.set('STEAM_ID', 'YOUR_STEAM_ID64')
""", file=sys.stderr)
        return

    output_filename = 'output_detailed.txt'
    sleep_duration = 1.6

    game_data = get_owned_games(api_key, steam_id)

    with redirect_stdout(output_filename):
        if game_data and "response" in game_data and "games" in game_data["response"]:
            games = game_data["response"]["games"]
            total_games = game_data["response"].get("game_count", len(games))
            print(f"\n--- 所有ゲームリスト (合計: {total_games} 件) ---")

            games = [game for game in games if game.get("playtime_forever", 0) > 0]
            filtered_count = total_games - len(games)
            if filtered_count > 0:
                print(f"(プレイ時間0分のゲーム {filtered_count} 件を非表示にしました)")

            # ソート
            games.sort(key=lambda x: x.get("rtime_last_played", 0), reverse=True)
            print("(最終プレイ日時順にソートしました)")

            print(f"詳細情報を取得するゲーム数: {len(games)} 件")
            print("App Details APIへの接続を開始します (レート制限のため時間がかかります)...")
            print("\n詳細情報の取得を開始します。進捗はファイルに出力されます...", file=sys.stderr)

            for i, game in enumerate(games):
                app_id = game.get("appid")
                if not app_id:
                    print(f"\n警告: {i+1}/{len(games)} - AppIDが見つからないためスキップします。ゲームデータ: {game}", file=sys.stderr)
                    continue

                name = game.get("name", f"不明 (AppID: {app_id})")
                playtime_hours = game.get("playtime_forever", 0) / 60
                last_played_formatted = format_timestamp(game.get("rtime_last_played"))

                app_details = get_app_details(app_id)

                genre_str = "情報なし"
                category_str = "情報なし"

                if app_details:
                    genres = app_details.get('genres', [])
                    genre_str = ", ".join([g.get("description") for g in genres if isinstance(g, dict) and g.get("description")])

                    categories = app_details.get('categories', [])
                    category_str = ", ".join([c.get("description") for c in categories if isinstance(c, dict) and c.get("description")])

                print(f"\n{i+1}/{len(games)}. {name}")
                print(f"   AppID: {app_id}")
                print(f"   総プレイ時間: {playtime_hours:.2f} 時間 ({game.get('playtime_forever', 0)} 分)")
                print(f"   最終プレイ: {last_played_formatted}")
                print(f"   ジャンル: {genre_str}")
                print(f"   カテゴリ: {category_str}")
                time.sleep(sleep_duration)

        elif game_data and "response" in game_data and not game_data["response"]:
            print("\nゲームリストを取得できませんでした。Steamプロフィールのゲーム詳細が非公開になっている可能性があります。", file=sys.stderr)
        elif game_data is None:
            print("\nGetOwnedGames APIからのデータ取得に失敗したため、処理を中断しました。エラーメッセージを確認してください。", file=sys.stderr)
        else:
            print("\nエラー: GetOwnedGames APIから予期しない形式のレスポンスを受け取りました。", file=sys.stderr)
            print(f"受信データ: {game_data}", file=sys.stderr)

        print("\n--- 処理終了 ---")

    print(f"\n処理が完了し、結果が {output_filename} に保存されました。", file=sys.stderr)
    print("処理中に警告やエラーが表示された場合は、コンソール出力も確認してください。", file=sys.stderr)

if __name__ == "__main__":
    main()

GetOwnedGames APIに接続中...



詳細情報の取得を開始します。進捗はファイルに出力されます...

処理が完了し、結果が output_detailed.txt に保存されました。
処理中に警告やエラーが表示された場合は、コンソール出力も確認してください。
